In [1]:
import requests
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

#header for API authorization
headers = {
    'Flic-Token': 'flic_6e2d8d25dc29a4ddd382c2383a903cf4a688d1a117f6eb43b35a1e7fadbb84b8'
}


# API Endpoints (Replace with your actual URLs)
VIEWED_POSTS_API = "https://api.socialverseapp.com/posts/view?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if"
LIKED_POSTS_API = "https://api.socialverseapp.com/posts/like?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if"
INSPIRED_POSTS_API = "https://api.socialverseapp.com/posts/inspire?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if"
RATED_POSTS_API = "https://api.socialverseapp.com/posts/rating?page=1&page_size=1000&resonance_algorithm=resonance_algorithm_cjsvervb7dbhss8bdrj89s44jfjdbsjd0xnjkbvuire8zcjwerui3njfbvsujc5if"
All_POSTS_API= 'https://api.socialverseapp.com/posts/summary/get?page=1&page_size=1000'
ALL_USER_API='https://api.socialverseapp.com/users/get_all?page=1&page_size=1000' 




# 1️⃣ Data Retrieval (API Calls)
def fetch_data(api_url):
    """Fetch data from API and return as DataFrame."""
    response = requests.get(api_url,headers=headers)
    if response.status_code == 200:
        return pd.DataFrame(response.json())
    else:
        print(f"Error fetching data from {api_url}")
        return pd.DataFrame()

# Fetch data from all APIs
# Fetch data from all APIs
viewed_posts = fetch_data(VIEWED_POSTS_API)
viewed_posts=pd.json_normalize(viewed_posts['posts'])
liked_posts = fetch_data(LIKED_POSTS_API)
liked_posts = pd.json_normalize(liked_posts['posts'])
inspired_posts = fetch_data(INSPIRED_POSTS_API)
inspired_posts= pd.json_normalize(inspired_posts['posts'])
rated_posts = fetch_data(RATED_POSTS_API)
rated_posts= pd.json_normalize(rated_posts['posts'])
all_post=fetch_data(All_POSTS_API)
all_post=pd.json_normalize(all_post['posts'])
all_post=all_post[['id','slug','title','identifier']]
all_user=fetch_data(ALL_USER_API)
all_user=pd.json_normalize(all_user['users'])
all_user=all_user[['id','username']]

def time_to_seconds(time_str):
    # Extract time part from 'YYYY-MM-DD HH:MM:SS' and split by ':'
    time_part = time_str.split()[1]
    h, m, s = map(int, time_part.split(':'))
    return h * 3600 + m * 60 + s

viewed_posts['view_time_seconds'] = viewed_posts['viewed_at'].apply(time_to_seconds)
liked_posts['like_time_seconds'] = liked_posts['liked_at'].apply(time_to_seconds)
inspired_posts['inspiration_time_seconds'] = inspired_posts['inspired_at'].apply(time_to_seconds)
rated_posts['rated_posts_time_seconds']=rated_posts['rated_at'].apply(time_to_seconds)



# Remove duplicates
viewed_posts.drop_duplicates(subset=['user_id', 'post_id'], inplace=True)
liked_posts.drop_duplicates(subset=['user_id', 'post_id'], inplace=True)
inspired_posts.drop_duplicates(subset=['user_id', 'post_id'], inplace=True)
rated_posts.drop_duplicates(subset=['user_id', 'post_id'], inplace=True)



# Normalize engagement scores
scaler = MinMaxScaler()
engagement_data = pd.merge(viewed_posts, liked_posts, on=['user_id', 'post_id','id'], how='outer')
engagement_data = pd.merge(engagement_data, inspired_posts, on=['user_id', 'post_id','id'], how='outer')
engagement_data = pd.merge(engagement_data, rated_posts, on=['user_id', 'post_id','id'], how='outer')
video_megadata=pd.merge(all_post, all_user, on='id', how='outer', suffixes=('_left', '_right'))
video_megadata = video_megadata.dropna()
engagement_data=pd.merge(engagement_data,video_megadata, on='id',how='outer')
engagement_data.fillna(0, inplace=True)

engagement_data.drop(columns=['viewed_at', 'liked_at', 'inspired_at','rated_at'], inplace=True)

engagement_data['user_id']=engagement_data['user_id'].astype(int)

# Calculate a unified engagement score
engagement_data['engagement_score'] = (
    (engagement_data['view_time_seconds'] * (0.3)) +
    (engagement_data['like_time_seconds'] * (0.3)) +
    (engagement_data['inspiration_time_seconds'].notnull().astype(int) * (0.2)) +
    (engagement_data['rated_posts_time_seconds'] * (0.2))
)

# Normalize engagement scores
engagement_data['normalized_score'] = scaler.fit_transform(engagement_data[['engagement_score']])

# Mood Calculation (custom logic)
engagement_data['mood_score'] = (engagement_data['like_time_seconds'] / engagement_data['view_time_seconds']) * engagement_data['rating_percent']
engagement_data['mood_score'].fillna(0, inplace=True)
engagement_data['mood'] = engagement_data['mood_score'].apply(lambda x: 'Positive' if x > 50 else 'Negative')


engagement_data



,id,post_id,user_id,view_time_seconds,like_time_seconds,inspiration_time_seconds,rating_percent,rated_posts_time_seconds,slug,title,identifier,username,engagement_score,normalized_score,mood_score,mood
0,18,631.0,114,26455.0,0.0,0.0,0.0,0.0,0,0,0,0,7936.7,0.306881,0.0,Negative
1,18,64.0,18,0.0,63123.0,0.0,0.0,0.0,0,0,0,0,18937.1,0.732234,0.0,Negative
2,18,469.0,1,0.0,0.0,36636.0,0.0,0.0,0,0,0,0,0.2,0.000000,0.0,Negative
3,18,517.0,1,0.0,0.0,0.0,82.0,63771.0,0,0,0,0,12754.4,0.493168,0.0,Negative
4,19,626.0,114,26478.0,0.0,0.0,0.0,0.0,we-are-made-of-stars,WE ARE MADE OF STARS,ATg3vKN,johntest,7943.6,0.307148,0.0,Negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3247,687,0.0,0,0.0,0.0,0.0,0.0,0.0,dc0f546db4616da5595779eadd4afb5efab21837,Do not fear I am with you,isW62Tj,vishnumurthy,0.2,0.000000,0.0,Negative
3248,702,0.0,0,0.0,0.0,0.0,0.0,0.0,671287dfb297579e1a6647017e36e66b077fc70c,Jai Shri Ram Manojavam Marutatulya,ckTMNyQ,ypk4my5sq9_01l1c4i8f,0.2,0.000000,0.0,Negative
3249,805,0.0,0,0.0,0.0,0.0,0.0,0.0,3a49f9ed74e466e135ccd73d473a77d133f37a38,first meet of shatrughan with love kush,Fom2XOH,kjy8frhg6t_08f2l3c8i,0.2,0.000000,0.0,Negative
3250,860,0.0,0,0.0,0.0,0.0,0.0,0.0,8143de1267f0cbcd19360c7fec83d220641e4cb9,Ramayan part 20,wS9HaNG,2r6m2grxsy_09l6c6f6i,0.2,0.000000,0.0,Negative


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Dataset, Reader, accuracy  # Import accuracy
from surprise.model_selection import train_test_split
import numpy as np

# Assuming 'engagement_data' is already loaded as a DataFrame
df = engagement_data  # Your engagement dataset

# Ensure 'title' is in string format for TF-IDF processing
df['title'] = df['title'].astype(str)

# Step 1: Content-based Filtering (TF-IDF)
def compute_content_scores(df, mood=None):
    """Compute content-based scores using TF-IDF."""
    tfidf = TfidfVectorizer(stop_words='english')
    tfidf_matrix = tfidf.fit_transform(df['title'])
    
    if mood:
        mood_based_post_idx = df[df['mood'].str.lower() == mood.lower()].index
        if not mood_based_post_idx.empty:
            content_scores = cosine_similarity(tfidf_matrix[mood_based_post_idx[0]], tfidf_matrix)
        else:
            content_scores = cosine_similarity(tfidf_matrix[0], tfidf_matrix)
    else:
        content_scores = cosine_similarity(tfidf_matrix[0], tfidf_matrix)
    
    return content_scores.ravel()  # Convert to 1D array

# Step 2: Collaborative Filtering (SVD)
def train_svd_model(df):
    """Train an SVD model for collaborative filtering and calculate MAE and RMSE."""
    reader = Reader(rating_scale=(df['engagement_score'].min(), df['engagement_score'].max()))
    data = Dataset.load_from_df(df[['user_id', 'post_id', 'engagement_score']], reader)
    
    trainset, testset = train_test_split(data, test_size=0.2)
    
    model = SVD()
    model.fit(trainset)
    
    predictions = model.test(testset)
    
    # Calculate MAE and RMSE
    mae = accuracy.mae(predictions, verbose=True)
    rmse = accuracy.rmse(predictions, verbose=True)
    
    return model, predictions, mae, rmse

# Step 3: Hybrid Model
def hybrid_recommendation(username, user_id, mood=None, alpha=0.6, beta=0.4):
    """
    Hybrid recommendation system that combines content-based and collaborative filtering.
    Returns:
        list: List of recommended post IDs for the user.
    """
    # Step 1: Content-based scores
    content_scores = compute_content_scores(df, mood=mood)
    df['content_score'] = content_scores  # Store the content scores in the DataFrame
    
    # Step 2: Collaborative Filtering
    model, predictions, mae, rmse = train_svd_model(df)  # Get MAE and RMSE here
    
    # Extract collaborative scores for the given user
    user_predictions = [pred for pred in predictions if pred.uid == str(user_id)]
    
    if not user_predictions:
        print(f"No predictions found for user_id {user_id}, using default predictions.")
        user_predictions = predictions  # Default to all predictions if user_id not found
    
    # Extract collaborative scores and align them with post IDs
    collaborative_df = pd.DataFrame({
        'post_id': [pred.iid for pred in user_predictions],
        'collaborative_score': [pred.est for pred in user_predictions]
    })
    
    # Merge collaborative scores with the main DataFrame
    merged_df = pd.merge(df, collaborative_df, on='post_id', how='left')
    
    # Replace NaN collaborative scores with 0
    merged_df['collaborative_score'].fillna(0, inplace=True)
    
    # Step 3: Calculate the hybrid final score
    merged_df['final_score'] = (alpha * merged_df['content_score']) + (beta * merged_df['collaborative_score'])
    
    # Step 4: Rank and recommend top 10 posts
    recommended_posts = merged_df.sort_values(by='final_score', ascending=False).head(10)
    
    # Return the recommended post IDs
    return recommended_posts['post_id'].tolist(), mae, rmse

# Example usage:
username = "vishnumurthy"  # User's name
user_id = 114 # Example user ID
mood = "positive"  # Example mood

recommended_posts, mae, rmse = hybrid_recommendation(username, user_id, mood)

# Output the top 10 recommended post IDs
if recommended_posts:
    print(f"Top 10 recommended post IDs for user '{username}' (user ID {user_id}) and mood '{mood}':")
    print(recommended_posts)



MAE:  15179.0771
RMSE: 16542.3982
No predictions found for user_id 114, using default predictions.
Top 10 recommended post IDs for user 'vishnumurthy' (user ID 114) and mood 'positive':
[631.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0, 33.0]
